In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install datasets
!pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
from datasets import load_dataset

# Load the dataset with the `large_random_1k` subset
dataset = load_dataset('poloclub/diffusiondb', '2m_first_1k')

In [3]:
my_1k_data = dataset['train']
my_1k_data

Dataset({
    features: ['image', 'prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw', 'prompt_nsfw'],
    num_rows: 1000
})

In [4]:
from huggingface_hub import login
login()

In [7]:
from transformers import AutoTokenizer
import transformers
import torch
import random
import json
import re

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

#testing for one index
index = 0

samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

sequences = pipeline(
    # samp + my_1k_data['prompt'][index] + "<Q&A>",
    samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
    # samp + my_1k_data['prompt'][index] ,
    # my_1k_data['prompt'][index],
    do_sample=False,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=200,
    max_new_tokens=512,
    temperature = 1e-05,
    top_p = 1,
    repetition_penalty = 1.0,
    length_penalty = 1
)

text = sequences[0]['generated_text']

input_text = text.split('Question-Answer pair to be generated:')
input_text

qa_dict = {}

# Iterate through array starting from the second index
for i in range(2, len(input_text), 2):
    # Extract questions and answers
    questions = re.findall(r'Q\..*?\?', input_text[i])
    answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

    # Populate the dictionary with questions and answers
    for question, answer in zip(questions, answers):
        qa_dict[question.strip()] = answer.strip()

# Print the resulting dictionary
print(qa_dict)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/amudra/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'Q. What is the subject of the portrait?': 'A. The subject of the portrait is Dwayne Johnson', 'Q. What is the style of the painting?': 'A. The painting is in the style of Rembrandt', 'Q. What is the medium used in the painting?': 'A. The medium used in the painting is oil on canvas', 'Q. What is the technique used in the painting?': 'A. The technique used in the painting is wet-on-wet', 'Q. What is the level of realism in the painting?': 'A. The level of realism in the painting is highly realistic', 'Q. What is the lighting effect in the painting?': 'A. The lighting effect in the painting is cinematic lightning', 'Q. What are the textures like in the painting?': 'A. The textures in the painting are intricate', 'Q. What is the level of detail in the painting?': 'A. The level of detail in the painting is highly intricate and illusionistic.'}


In [ ]:
samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

# Initialize the final output dictionary
final_output = {}

for index in range(min(10, len(my_1k_data['image']))):

    sequences = pipeline(
        samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
        do_sample=False,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        temperature=1e-05,
        top_p=1,
        repetition_penalty=1.0,
        length_penalty=1
    )

    text = sequences[0]['generated_text']

    input_text = text.split('Question-Answer pair to be generated:')
    
    qa_dict = {}
    for i in range(2, len(input_text), 2):
        questions = re.findall(r'Q\..*?\?', input_text[i])
        answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

        for question, answer in zip(questions, answers):
            qa_dict[question.strip()] = answer.strip()

    final_output[index] = {
        'image': my_1k_data['image'][index],
        'prompt': my_1k_data['prompt'][index],
        'qa_pairs': qa_dict
    }

print(final_output)


In [ ]:
######################################################################

In [ ]:
#####################################################################

In [ ]:
#Final Single Cell Code for QnA Gen of All Images in Dataset
#Can be implemented after Cell 4
#Input: my_1k_data (dataset)
#Output: final_output (dictionary)

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import random
import json
import re

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

# Initialize the final output dictionary
final_output = {}

for index in range(len(my_1k_data['image'])):

    sequences = pipeline(
        samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
        do_sample=False,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        temperature=1e-05,
        top_p=1,
        repetition_penalty=1.0,
        length_penalty=1
    )

    text = sequences[0]['generated_text']

    input_text = text.split('Question-Answer pair to be generated:')
    
    qa_dict = {}
    for i in range(2, len(input_text), 2):
        questions = re.findall(r'Q\..*?\?', input_text[i])
        answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

        for question, answer in zip(questions, answers):
            qa_dict[question.strip()] = answer.strip()

    final_output[index] = {
        'image': my_1k_data['image'][index],
        'prompt': my_1k_data['prompt'][index],
        'qa_pairs': qa_dict
    }

print(final_output)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]